# move data from one db to another

see: https://github.com/APS-3ID-IXN/ipython-s3blue/issues/2

the packages we'll be using

In [1]:
from databroker import Broker
import datetime, os, shutil, time
import pyRestTable

connect to the database where scans were collected

In [2]:
server = "otz"
otz_config = {
    "description": "heavyweight shared database",
    "metadatastore": {
        "module": "databroker.headersource.mongo",
        "class": "MDS",
        "config": {
            "host": server,
            "port": 27017,
            "database": "metadatastore-production-v1",
            "timezone": "US/Central"
        }
    },
    "assets": {
        "module": "databroker.assets.mongo",
        "class": "Registry",
        "config": {
            "host": server,
            "port": 27017,
            "database": "filestore-production-v1"
        }
    }
}
otz_db = Broker.from_config(otz_config)

Show the scans between the dates mentioned in the [issue](https://github.com/APS-3ID-IXN/ipython-s3blue/issues/2)

In [3]:
headers = otz_db(since='2018-09-01', until='2018-11-06')
for h in headers:
    print(h.start["beamline_id"], h.start["login_id"], h.start["plan_name"])

3-ID s3blue@ni.xray.aps.anl.gov rel_grid_scan
3-ID s3blue@ni.xray.aps.anl.gov rel_grid_scan
3-ID s3blue@ni.xray.aps.anl.gov rel_grid_scan
3-ID s3blue@ni.xray.aps.anl.gov rel_grid_scan
3-ID s3blue@ni.xray.aps.anl.gov rel_grid_scan
3-ID s3blue@ni.xray.aps.anl.gov rel_grid_scan
3-ID s3blue@ni.xray.aps.anl.gov rel_grid_scan
3-ID s3blue@ni.xray.aps.anl.gov rel_grid_scan
3-ID s3blue@ni.xray.aps.anl.gov rel_grid_scan
3-ID s3blue@ni.xray.aps.anl.gov rel_grid_scan
3-ID s3blue@ni.xray.aps.anl.gov rel_grid_scan
3-ID s3blue@ni.xray.aps.anl.gov rel_grid_scan
3-ID s3blue@ni.xray.aps.anl.gov rel_grid_scan
3-ID s3blue@ni.xray.aps.anl.gov rel_grid_scan
3-ID s3blue@ni.xray.aps.anl.gov rel_grid_scan
3-ID s3blue@ni.xray.aps.anl.gov rel_grid_scan
3-ID s3blue@ni.xray.aps.anl.gov rel_grid_scan
3-ID s3blue@ni.xray.aps.anl.gov rel_grid_scan
3-ID s3blue@ni.xray.aps.anl.gov rel_grid_scan
3-ID s3blue@ni.xray.aps.anl.gov rel_grid_scan
3-ID s3blue@ni.xray.aps.anl.gov rel_grid_scan
3-ID s3blue@ni.xray.aps.anl.gov re

All those scans have the key `beamline_id="3-ID"` so get all the scan (headers) that match that key.  Print the results in a nicer table.

In [4]:
tbl = pyRestTable.Table()
tbl.labels = "date/time # uid plan_name".split()
headers = otz_db(beamline_id='3-ID')
for h in headers:
    t_float = h.start["time"]
    dt = datetime.datetime.fromtimestamp(t_float)
    row = [dt, h.start["scan_id"], h.start["uid"], h.start["plan_name"]]
    tbl.addRow(row)
print(tbl)

========================== == ==================================== =============
date/time                  #  uid                                  plan_name    
========================== == ==================================== =============
2018-10-29 03:45:24.167806 48 6ea22d45-7b57-4936-b007-a6129a04a28c rel_grid_scan
2018-10-28 18:56:00.852009 47 b7f9a22f-bff3-4fca-8223-3fe3c13cf04a rel_grid_scan
2018-10-28 18:44:26.594428 46 ab52f840-7919-4925-8c62-790a0bf629ef rel_grid_scan
2018-10-28 16:59:11.531815 45 8b318daa-c3ee-46d3-9d42-15c51f2e7598 rel_grid_scan
2018-10-28 16:53:30.997551 44 a4b7fd59-7875-489a-8a5f-1cc44f311760 rel_grid_scan
2018-10-28 16:50:59.680815 43 8ee33a6f-0a74-43fa-8d1d-99e94e868811 rel_grid_scan
2018-10-28 16:48:00.945385 42 ed1ee892-61a3-4b8a-bb3d-4b108465a9e0 rel_grid_scan
2018-10-28 16:37:31.415016 41 7769bca7-bccd-42de-8459-9676d7dcde86 rel_grid_scan
2018-10-28 16:31:16.187840 40 57e0b82d-939b-4547-8eff-9eaad0401712 rel_grid_scan
2018-10-28 16:28:51.544602 3

-----------

## build a local sqlite database for testing

In [13]:
test_dir = "/tmp/bluesky"
if not os.path.exists(test_dir):
    os.makedirs(test_dir)

test_config = {
    "description": "lightweight personal database",
    "metadatastore": {
        "module": "databroker.headersource.sqlite",
        "class": "MDS",
        "config": {
            "directory": test_dir,
            "timezone": "US/Central"
        }
    },
    "assets": {
        "module": "databroker.assets.sqlite",
        "class": "Registry",
        "config": {
            "dbpath": test_dir + "/database.sql"
        }
    }
}
test_db = Broker.from_config(test_config)

## Export (test)
export from otz_db to test_db

In [14]:
if os.path.exists(test_dir):
    shutil.rmtree(test_dir)
    os.makedirs(test_dir)

for h in otz_db.stream(headers):
    tag, doc = h
    print(tag)
    test_db.insert(tag, doc)

start
descriptor


OperationalError: duplicate column name: data_NFS

In [ ]:
h = list(headers)[-1]
h.descriptors

Seems be trouble creating ophyd object names with the data column names that have embedded whitespace.  These names come from the text in the scaler channel labels.  It was OK to acquire data with such names.  

**Looks like an assumption bug to me in the export code.**  Either of these places:

* `databroker/headersource/core.py`
* `databroker/headersource/sqlite.py`

In [ ]:
list(h.events())[-1]